In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

from keras.layers import Conv2D, MaxPooling2D, Convolution2D, Dropout, Dense, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [0]:
img = np.load("./npy_files/img_"+mode+".npy")/255.
tar = np.load("./npy_files/tar_train_binary.npy")

img_test = np.load("./npy_files/img_test.npy")/255.
tar_test = np.load("./npy_files/tar_test_binary.npy")

In [0]:
input_shape = (28, 28, 1)
nb_classes = 2

model = Sequential()
model.add(Convolution2D(64, (5, 5), padding='valid', input_shape=input_shape))
model.add(LeakyReLU(0.01))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(BatchNormalization())
model.add(Convolution2D(64, (3, 3)))
model.add(LeakyReLU(0.01))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(BatchNormalization())
model.add(Dense(1024))
model.add(LeakyReLU(0.01))
model.add(Dropout(0.25))

model.add(Dense(nb_classes, activation='softmax'))

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# model.summary()

In [0]:
batch_size = 128
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3, height_shift_range=0.08, zoom_range=0.08)
val_gen = ImageDataGenerator()

train_generator = gen.flow(img, tar, batch_size=batch_size)
test_generator = val_gen.flow(img_test, tar_test, batch_size=batch_size)

t1 = img.shape[0]//batch_size
t2 = img_test.shape[0]//batch_size

In [0]:
try:
  history = model.fit_generator(train_generator, steps_per_epoch=t1, epochs=50, validation_data=test_generator, validation_steps=t2)
except KeyboardInterrupt as e:
  pass

In [0]:
model.save("./models/model1.h5")

In [81]:
model = keras.models.load_model("./models/model1.h5")

print("------------\nTesting acc")
print(model.evaluate(img_test, tar_test))
print("------------\nTraining acc")
print(model.evaluate(img, tar))

------------
Testing acc
18800/18800 [==============================] - 2s 90us/step
[0.13660382586987094, 0.935372340425532]
------------
Training acc
112800/112800 [==============================] - 9s 81us/step
[0.10434728366687542, 0.9461790780141844]


In [116]:
idx = np.random.randint(0, 18000)

vals = ["letter", "number"]

image = np.reshape(img_test[idx], (1, 28, 28, 1))
prediction = model.predict_classes(image)

print("Prediction: %s"%vals[prediction[0]])
print("Actual val: %s"%vals[np.argmax(tar_test[idx])])

cv2.imshow(image[0,:,:,0]*255.)

Prediction: number
Actual val: number


In [112]:
# CLASS-WISE PREDICTION RESULTS

Y_test = np.argmax(tar_test, axis=1)
y_pred = model.predict_classes(img_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96     14800
           1       0.82      0.89      0.85      4000

    accuracy                           0.94     18800
   macro avg       0.90      0.92      0.91     18800
weighted avg       0.94      0.94      0.94     18800

